In [ ]:
pip install sentencepiece


In [ ]:
pip install rouge-score


In [ ]:
import os
from transformers import pipeline
from rouge_score import rouge_scorer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder='/content/drive/My Drive/DATA2011/'
!ls '/content/drive/My Drive/DATA2011/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
HumSum	sourceTexts


In [ ]:
translation_grk_to_en = pipeline("translation_grk_to_en", model='Helsinki-NLP/opus-mt-grk-en')
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
translation_en_to_el = pipeline("translation_en_to_el", model='Helsinki-NLP/opus-mt-en-el')

In [ ]:
#split text into smaller chunks
def read_and_split_text(file_path, chunk_size=250):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    chunks = []
    current_chunk = ""
    words_count = 0
    for line in text.splitlines():
        line_words = len(line.split())
        if words_count + line_words > chunk_size and current_chunk:
            chunks.append(current_chunk)
            current_chunk = line + '\n'
            words_count = line_words
        else:
            current_chunk += line + '\n'
            words_count += line_words
    if current_chunk:
        chunks.append(current_chunk)
    return chunks


# Process each chunk through the pipelines
def process_chunks(chunks):
    translated_chunks = [translation_grk_to_en(chunk)[0]['translation_text'] for chunk in chunks]
    summarized_chunks = []
    for chunk in translated_chunks:
        summarized_chunk = summarizer(chunk, max_length=512, min_length=30, length_penalty=2.0)[0]['summary_text']
        summarized_chunks.append(summarized_chunk)
    retranslated_chunks = [translation_en_to_el(chunk)[0]['translation_text'] for chunk in summarized_chunks]
    return "\n".join(retranslated_chunks)

# read golden summaries
def read_human_summary(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# process files  and perform ROUGE
def process_and_evaluate(folder_path, human_summary_folder):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    print(f"Processing files in folder: {folder_path}")

    for filename in os.listdir(folder_path):
        if filename.endswith(".greek"):
            print(f"Processing file: {filename}")
            input_file_path = os.path.join(folder_path, filename)
            human_summary_path = os.path.join(human_summary_folder, filename.replace('.greek', '.ref'))


            chunks = read_and_split_text(input_file_path)
            machine_summary = process_chunks(chunks)

            print(f"Summarized text for {filename}:\n{machine_summary}\n")


            human_summary = read_human_summary(human_summary_path)

            # Calculate ROUGE scores
            scores = scorer.score(human_summary, machine_summary)

            print(f"ROUGE scores for {filename}:\n{scores}\n")


folder_path = folder+'sourceTexts'
human_summary_folder = folder+'HumSum'

process_and_evaluate(folder_path, human_summary_folder)


Processing files in folder: /content/drive/My Drive/DATA2011/sourceTexts
Processing file: M0014.greek


Your max_length is set to 512, but your input_length is only 157. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=78)
Your max_length is set to 512, but your input_length is only 187. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=93)
Your max_length is set to 512, but your input_length is only 203. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=101)
Your max_length is set to 512, but your input_length is only 174. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=87)

Summarized text for M0014.greek:
52 άνθρωποι σκοτώθηκαν και εκατοντάδες τραυματίστηκαν στις 7 Ιουλίου 2005, όταν τέσσερις βομβιστές αυτοκτονίας ανατινάχτηκαν σε τρεις διαφορετικές άμαξες του υπόγειου Λονδίνου και σε ένα λεωφορείο.
Κα Νορθ: "Τρέχουμε το ρίσκο στα τρένα, στους δρόμους κάθε μέρα" "Πώς τολμάς να νομίζεις ότι ξέρεις τις ερωτήσεις μας," λέει. "Ξέρω τι συνέβη, θέλω να ξέρω γιατί"
"Όταν είδα το βίντεο της Αλ Κάιντα, όπου ειπώθηκε ότι η Τζιχάντ πάλευε εναντίον του Ηνωμένου Βασιλείου ανησυχώ από το πόσο εξοικειωμένη ήταν η φωνή μου," λέει. "Οι λέξεις που είπε ήταν παρόμοιες με εκείνες εκείνων που έχουν ακούσει πολλές φορές για το κακό των υπόλοιπων νέων"
Μια αφήγηση των γεγονότων δεν θα είναι αρκετή για κανέναν, λέει η Behnaz Mozakka. "Αναρωτιέμαι - αν υπάρχει κάτι που πρέπει να κρατηθεί κρυφό, γιατί να αποφευχθεί η δημόσια έρευνα," αναφέρει.

ROUGE scores for M0014.greek:
{'rouge1': Score(precision=0.6, recall=0.6, fmeasure=0.6), 'rouge2': Score(precision=0.25, recall=0.25, fme

Your max_length is set to 512, but your input_length is only 138. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)
Your max_length is set to 512, but your input_length is only 176. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=88)
Your max_length is set to 512, but your input_length is only 177. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=88)
Your max_length is set to 512, but your input_length is only 79. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)


Summarized text for M0020.greek:
Η ιρανική κυβέρνηση δήλωσε αρχικά ότι η τοποθεσία του συμβάντος δεν ήταν η πρώτη φορά που τα βρετανικά πλοία είχαν εισέλθει στα ύδατα του Ιράν. Μετά από σχόλια της βρετανικής κυβέρνησης, το Ιράν δήλωσε μια αναθεωρημένη τοποθεσία. " νέα τοποθεσία που δόθηκε από τους Ιρανούς ήταν εκτός από τα ιρακινά και εντός των ιρανικών χωρικών υδάτων.
Το περιστατικό έλαβε χώρα στην περιοχή Ras al-Beesha κοντά στη νότια είσοδο του Κόλπου. " βρετανική κυβέρνηση ζήτησε την επιστροφή των στρατιωτών. Ο Ιρανός πρέσβης προσκλήθηκε στο Υπουργείο Εξωτερικών.
Οκτώ μέλη του Βασιλικού Ναυτικού συνελήφθησαν από ιρανικές δυνάμεις τον Ιούνιο του 2004.
Ο Ρέζα Φάκερ, συγγραφέας ο οποίος φέρεται να έχει δεσμούς με τον Πρόεδρο Αχμαντινετζάντ, έγραψε στον Σουμπί Σάντεκ (η εβδομαδιαία εφημερίδα της Επαναστατικής Φρουράς) "Έχουμε τη δυνατότητα να συλλάβουμε μια καλή ομάδα Γαλάνικων ξανθών αξιωματούχων και να τους ταΐζουμε στους κοκκινωπούς μαχητές μας," έγραψε.

ROUGE scores for M0020.gree

...

Summarized text for M0067.greek:
Η αστυνομία της Αϊτής συνέλαβε δέκα Αμερικανούς, πέντε άνδρες και πέντε γυναίκες για δήθεν απαγωγή 33 παιδιών.
Η Αϊτινή πάστορας Ζαν Σανμπίλ, που μοιράζεται τα Υπουργεία του Ιησού, τους επέτρεψε να πάρουν τα παιδιά.
Η ομάδα NCLR σχεδίαζε να μεταφέρει περίπου 100 παιδιά στη Δομινικανή Δημοκρατία. Η πρόθεση των Βαπτιστών ήταν "να βρουν μια θεραπεία, ελπίδα, χαρά και μια νέα ζωή στο Χριστό" για τα Ρωμαιοκαθολικά παιδιά.
Τα παιδιά έχουν μεταφερθεί σε ένα χωριό SOS στο Σάντο, βόρεια της πρωτεύουσας. Το προσωπικό είπε ότι τα παιδιά ήταν σε πολύ κακή συναισθηματική κατάσταση.

ROUGE scores for M0067.greek:
{'rouge1': Score(precision=0.25, recall=0.3333333333333333, fmeasure=0.28571428571428575), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.25, recall=0.3333333333333333, fmeasure=0.28571428571428575)}

Processing file: M0079.greek


Your max_length is set to 512, but your input_length is only 137. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=68)
Your max_length is set to 512, but your input_length is only 202. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=101)
Your max_length is set to 512, but your input_length is only 159. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=79)


Summarized text for M0079.greek:
Ο πρώην διοικητής του Abu Gribb, ταγματάρχης David Dinena, δήλωσε ότι τον Σεπτέμβριο του 2003 παρευρέθηκε σε συνάντηση με τον Στρατηγό Geoffrey D. Miller. Ταγματάρχης Dinena είπε ότι ο Ταγματάρχης Miller πρότεινε τη χρήση σκύλων, λόγω της αποτελεσματικότητάς τους.
Ο αρμόδιος δικαστής του δικαστηρίου, ο Ταγματάρχης Γκλεν Σίμπκινς, έχει δύο εβδομάδες για να μελετήσει τα αποδεικτικά στοιχεία που παρουσιάστηκαν.
Ο Μήμος από το 2003 ενημέρωσε το Πεντάγωνο για τις βίαιες τακτικές που ακολουθήθηκαν στο Γκουαντάναμο. Νομοθεσία που προτάθηκε από κάποιους Δημοκρατικούς στη Γερουσία επιχειρεί να θεσπίσει ειδικούς κανόνες για τη μεταχείριση των κρατουμένων.

ROUGE scores for M0079.greek:
{'rouge1': Score(precision=0.5454545454545454, recall=0.6, fmeasure=0.5714285714285713), 'rouge2': Score(precision=0.3, recall=0.3333333333333333, fmeasure=0.3157894736842105), 'rougeL': Score(precision=0.5454545454545454, recall=0.6, fmeasure=0.5714285714285713)}

Processing file:

Your max_length is set to 512, but your input_length is only 135. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=67)
Your max_length is set to 512, but your input_length is only 135. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=67)
Your max_length is set to 512, but your input_length is only 113. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=56)


Summarized text for M0093.greek:
Ολυμπιακή λάμπα στο Σαν Φρανσίσκο της Καλιφόρνιας. Αυτή είναι η μόνη στάση της φλόγας στις Ηνωμένες Πολιτείες, κατά τη διάρκεια του ταξιδιού της σε όλο τον κόσμο, μέχρι να τελειώσει στο Πεκίνο.
"Είναι μια μοναδική ευκαιρία για εμάς να δούμε τη φλόγα," είπε στο Σαν Φρανσίσκο Chronicle, Nancy Chan, μια τοπική κάτοικος. "Είμαι περήφανος που είμαι ένας Κινέζος γιος στη νέα διαδρομή, μόλις ανακοίνωσε," δήλωσε ο Xiao Tan, 32.
Η Ακτοφυλακή των Ηνωμένων Πολιτειών έχει κάνει περιπολίες στο παράκτιο τμήμα του Σαν Φρανσίσκο. Τα αυστηρά μέτρα ασφαλείας έχουν σχεδιαστεί εκ των προτέρων. "Εδώ προσπαθούμε να επιτύχουμε δύο στόχους.Ο ένας είναι να προστατεύσουμε το δικαίωμα στην ελευθερία της έκφρασης," λέει ο Νέιθαν Μπάλαρντ.

ROUGE scores for M0093.greek:
{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}



In [ ]:
print(f"ROUGE scores for {filename}:\n{scores}\n")

NameError: name 'filename' is not defined